<a href="https://colab.research.google.com/github/Kavishankarks/Forecasting_realtime_trend/blob/main/MultivariateLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytrends
import plotly.express as px # to plot the time series plot
from sklearn import metrics # for the evaluation
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
import tensorflow as tf
from pandas import DataFrame
from pytrends.request import TrendReq
import pandas as pd
import time
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
# keywords=['deep learning','machine', 'python', 'network', 'image', 'ai', 'neural','reinforcement','google']
keywords=['pes','University','vaccine','pfizer','covid','appointment', 'astrazeneca','covishield','moderna']
dfk=DataFrame(keywords,columns=['Keywords'])
dfk.reset_index(drop=True, inplace=True)
dfk.to_csv('keywords.csv')

In [ ]:
def genData(date):
  from pytrends.request import TrendReq
  import pandas as pd
  import time
  startTime = time.time()
  pytrend = TrendReq(hl='en-GB', tz=360)

  colnames = ["keywords"]
  df = pd.read_csv("keywords.csv", names=colnames)
  df2 = df["keywords"].values.tolist()
  df2.remove("Keywords")

  dataset = []

  for x in range(0,len(df2)):
      keywords = [df2[x]]
      pytrend.build_payload(
      kw_list=keywords,
      cat=0,
      timeframe=date,geo='GB')
      data = pytrend.interest_over_time()
      if not data.empty:
            data = data.drop(labels=['isPartial'],axis='columns')
            dataset.append(data)

  result = pd.concat(dataset, axis=1)
  result.to_csv('trends.csv')

  executionTime = (time.time() - startTime)
  print('Execution time in sec.: ' + str(executionTime))
genData('2004-01-01 2021-10-10')
data=pd.read_csv('trends.csv')

In [ ]:
data=pd.read_csv('trends.csv')
data.head(10)

In [ ]:
res=['hel']
a='pes university'
a=a.split()
for i in a:
  res.insert(0,i)
# res.extend(a)
res

In [ ]:
def custom_ts_multi_data_prep(dataset, target, start, end, window, horizon):
    X = []
    y = []
    start = start + window
    if end is None:
        end = len(dataset) - horizon
    for i in range(start, end):
        indices = range(i-window, i)
        X.append(dataset[indices])
        indicey = range(i+1, i+1+horizon)
        y.append(target[indicey])
    return np.array(X), np.array(y)

In [ ]:
a=[]
for i in data.columns:
    a.append(i)
print(a)
for i in data.select_dtypes('object').columns:
    le = LabelEncoder().fit(data[i])
    data[i] = le.transform(data[i])                         
name=a[1]
X_scaler = MinMaxScaler()
Y_scaler = MinMaxScaler()
X_data = X_scaler.fit_transform(data[a])
Y_data = Y_scaler.fit_transform(data[[name]])

In [ ]:
len(data)

In [ ]:
validate = data[a[1]].tail(10)
# data.drop(data.tail(10).index,inplace=True)

In [ ]:
hist_window = 3
horizon = 7
TRAIN_SPLIT = 150
x_train, y_train = custom_ts_multi_data_prep(X_data, Y_data, 0, TRAIN_SPLIT, hist_window, horizon)
x_vali, y_vali = custom_ts_multi_data_prep(X_data, Y_data, TRAIN_SPLIT, None, hist_window, horizon)
print ('Multiple window of past history\n')
print(x_train[0])
print ('\n Target horizon\n')
print (y_train[0])

In [ ]:
# Prepare the training data and validation data using 
# the TensorFlow data function, which faster and efficient way to feed data for training.
batch_size = 256
buffer_size = 150
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.cache().shuffle(buffer_size).batch(batch_size).repeat()
val_data = tf.data.Dataset.from_tensor_slices((x_vali, y_vali))
val_data = val_data.batch(batch_size).repeat() 

# Build and compile the model
lstm_model = tf.keras.models.Sequential([
   tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(200, return_sequences=True), 
                                input_shape=x_train.shape[-2:]),
     tf.keras.layers.Dense(20, activation='tanh'),
     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150)),
     tf.keras.layers.Dense(20, activation='tanh'),
     tf.keras.layers.Dense(20, activation='tanh'),
     tf.keras.layers.Dropout(0.25),
     tf.keras.layers.Dense(units=horizon),
 ])
lstm_model.compile(optimizer='adam', loss='mse')
lstm_model.summary() 

In [ ]:
# Configure the model and start training with early stopping and checkpoint.
# Early stopping stops training when monitored loss starts increasing above the patience,
# and checkpoint saves the model weight as it reaches the minimum loss.
model_path = 'Bidirectional_LSTM_Multivariate.h5'
early_stopings = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='min')
checkpoint =  tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, mode='min', verbose=0)
callbacks=[early_stopings,checkpoint]

In [ ]:
history = lstm_model.fit(train_data,epochs=10,steps_per_epoch=5,validation_data=val_data,validation_steps=50,verbose=1,callbacks=callbacks)

In [ ]:
plt.figure(figsize=(16,9))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train loss', 'validation loss'])
plt.show()

In [ ]:
data_val = X_scaler.fit_transform(data[a].tail(10))
val_rescaled = data_val.reshape(1, data_val.shape[0], data_val.shape[1])
pred = lstm_model.predict(val_rescaled)
pred_Inverse = Y_scaler.inverse_transform(pred)
pred_Inverse

In [ ]:
name=a[1]
def timeseries_evaluation_metrics_func(y_true, y_pred):
  def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
  print('Evaluation metric results:-')
  print(len(y_true),len(y_pred))
  print(f'MSE is : {metrics.mean_squared_error(y_true, y_pred)}')
  print(f'MAE is : {metrics.mean_absolute_error(y_true, y_pred)}')
  print(f'RMSE is : {np.sqrt(metrics.mean_squared_error(y_true, y_pred))}')
  print(f'MAPE is : {mean_absolute_percentage_error(y_true, y_pred)}')
  print(f'R2 is : {metrics.r2_score(y_true, y_pred)}',end='\n\n')
timeseries_evaluation_metrics_func(validate[:7],pred_Inverse[0])
print(len(pred_Inverse[0]))

In [ ]:
name=a[1]
plt.figure(figsize=(16,9))
plt.plot(list(validate))
plt.plot(list(pred_Inverse[0]))
plt.title("Actual vs Predicted")
plt.ylabel(name)
plt.legend(('Actual','predicted'))
plt.show()